This example is an MB-MVPA code that uses part of the data (Piva et al., 2019).<br>
Piva's full dataset used 20 subjects, but in this example, only 2 subjects are used to save resource.

### Import the MB-MVPA libarary.

Other libraries(nilean, keras, etc..) dosen't need to be imported.<br>
Because mb-mvpa has wrapping the libararies.<br>
You don't necessarily have to know fMRI libraries like nilearn and machine learning libraries like tensorflow.<br>
<b>MB-MVPA is all you need.<b>

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from mbmvpa.preprocessing.bids import bids_preprocess
from mbmvpa.preprocessing.events import events_preprocess
from mbmvpa.data.loader import prepare_dataset
from mbmvpa.utils.example_utils import load_example_data

Data download from AWS S3, ~ <b>10GB</b>.<br>
If you run this example more than once, the data will already be downloaded.<br>
If So, do not download it again.

In [3]:
root = load_example_data('piva')

 40%|███▉      | 4.04G/10.2G [18:48<14:28, 7.12MiB/s]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Preprocessing bids data including fMRI images

In [4]:
X, voxel_mask, layout, data_root = bids_preprocess(root, smoothing_fwhm=None, zoom=(2, 2, 2), ncore=2, nthread=2)

bids preprocessing done!                          : 100%|██████████| 6/6 [02:12<00:00, 22.15s/it]  


### Preprocessing event data

In [5]:
def example_piva_adjust_columns(row):
    ## rename data in a row to the name which can match hbayesdm.dd_hyperbolic requirements ##
    if row["delay_left"] >= row["delay_right"]:
        row["delay_later"] = row["delay_left"]
        row["delay_sooner"] = row["delay_right"]
        row["amount_later"] = row["money_left"]
        row["amount_sooner"] = row["money_right"]
        row["choice"] = 1 if row["choice"] == 1 else 0
    else:
        row["delay_later"] = row["delay_right"]
        row["delay_sooner"] = row["delay_left"]
        row["amount_later"] = row["money_right"]
        row["amount_sooner"] = row["money_left"]
        row["choice"] = 1 if row["choice"] == 2 else 0
    return row

In [6]:
def example_piva_condition(row):
    # in the paper, the condition for trial varies in a single run,
    # agent == 0 for making a choice for him or herself
    # agent == 1 for making a choice for other
    # to consider only non-social choice behavior, select only the cases with agent == 0
    return row["agent"] == 0

In [7]:
def example_piva_modulation(row, param_dict):
    # calculate subjective utility for choosing later option over sooner option
    # hyperbolic discount function is adopted
    ev_later = row["amount_later"] / (1 + param_dict["k"] * row["delay_later"])
    ev_sooner  = row["amount_sooner"] / (1 + param_dict["k"] * row["delay_sooner"])
    modulation = ev_later - ev_sooner
    row["modulation"] = modulation
    return row

In [8]:
dm_model, df_events, signals, time_masks, _ = \
    events_preprocess(layout=layout,
                      preprocess=example_piva_adjust_columns,
                      condition=example_piva_condition,
                      modulation=example_piva_modulation,
                      dm_model="dd_hyperbolic")

hbayesdm doing (model: dd_hyperbolic)..                      :  57%|█████▋    | 4/7 [00:01<00:01,  2.00it/s]INFO:numexpr.utils:Note: detected 88 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 88 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:pystan:COMPILING THE C++ CODE FOR MODEL dd_hyperbolic_015c9b5212519ef6e8d499f3a5e46f5a NOW.



Model  = dd_hyperbolic
Data   = <pandas.DataFrame object>

Details:
 # of chains                    = 4
 # of cores used                = 1
 # of MCMC samples (per chain)  = 4000
 # of burn-in samples           = 1000
 # of subjects                  = 2
 # of (max) trials per subject  = 60

Using cached StanModel: cached-dd_hyperbolic-pystan_2.19.1.1.pkl


calculating modulation..                          :  71%|███████▏  | 5/7 [01:33<00:28, 14.16s/it]           

Index(['onset', 'duration', 'choice', 'agent', 'moneyleft', 'delayleft',
       'moneyright', 'delayright', 'subjid', 'run', 'session', 'delaylater',
       'delaysooner', 'amountlater', 'amountsooner'],
      dtype='object')
['onset', 'duration', 'choice', 'agent', 'money_left', 'delay_left', 'money_right', 'delay_right', 'subjID', 'run', 'session', 'delay_later', 'delay_sooner', 'amount_later', 'amount_sooner']
************************************
**** Model fitting is complete! ****
************************************


events preproecssing done!                        : 100%|██████████| 7/7 [01:34<00:00, 13.43s/it]


### Load data and shape check

In [9]:
X, y, voxel_mask = prepare_dataset(data_root)

### Fitting MVPA models & Results

In [10]:
from mbmvpa.utils.coef2map import get_map
from time import perf_counter

In [11]:
from mbmvpa.models.regressor import penalized_linear_regression

s = perf_counter()
coefs = penalized_linear_regression(X, y,
                                    layout,
                                    lambda_param=2.0,
                                    N=10,
                                    verbose=1)
result = get_map(coefs, voxel_mask, task_name="piva2019_penalized_linear", map_type="z", save_path=".", sigma=1)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

INFO:root:[1/10] - val_mse: 0.5020
INFO:root:[2/10] - val_mse: 0.5368
INFO:root:[3/10] - val_mse: 0.5216
INFO:root:[4/10] - val_mse: 0.5296
INFO:root:[5/10] - val_mse: 0.5306
INFO:root:[6/10] - val_mse: 0.5685
INFO:root:[7/10] - val_mse: 0.5727
INFO:root:[8/10] - val_mse: 0.4863
INFO:root:[9/10] - val_mse: 0.5197
INFO:root:[10/10] - val_mse: 0.5209


elapsed time: 1.48 minutes


In [ ]:
from mbmvpa.models.regressor import mlp_regression

s = perf_counter()
coefs = mlp_regression(X, y,
                       layout,
                       layer_dims=[1024, 1024],
                       activation="linear",
                       dropout_rate=0.5,
                       epochs=100,
                       patience=10,
                       batch_size=64,
                       N=3,
                       verbose=1)
result = get_map(coefs, voxel_mask, task_name="piva2019_mlp", map_type="z", save_path=".", sigma=1)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

In [ ]:
from mbmvpa.models.regressor import elasticnet

s = perf_counter()
coefs = elasticnet(X, y,
                   layout,
                   alpha=0.005,
                   n_jobs=16,
                   verbose=1,
                   max_lambda=1,
                   n_samples=5000)
result = get_map(coefs, voxel_mask, task_name="piva2019_elasticnet", map_type="z", save_path=".", sigma=1)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")